In [97]:
import base64
import json
import requests
import os
from dotenv import load_dotenv
from mypy_extensions import TypedDict
from typing import Dict, List
from requests.auth import HTTPBasicAuth
from dateutil import parser

load_dotenv('../.env')

user_key = os.environ.get('METRC_USER_KEY')
vendor_key_CA = os.environ.get('METRC_VENDOR_KEY_CA')

if not user_key or not vendor_key_CA:
    raise Exception('METRC_USER_KEY or METRC_VENDOR_KEY_CA not set')

auth_dict = {'vendor_key': vendor_key_CA, 'user_key': user_key}


In [98]:
licenses = [
    'CCL19-0001462',
]

class REST(object):
    
    def __init__(self, auth_dict, license_id):
        self.auth = HTTPBasicAuth(auth_dict['vendor_key'], auth_dict['user_key'])
        self.license_id = license_id
        self.base_url = 'https://api-ca.metrc.com'
        
    def get(self, path, time_range=None):
        url = self.base_url + path
        url += '?licenseNumber=' + self.license_id
        
        if time_range:
            if len(time_range) == 1:
                lastModifiedStart = parser.parse(time_range[0]).isoformat()
                url += '&lastModifiedStart=' + lastModifiedStart                
            else:
                lastModifiedStart = parser.parse(time_range[0]).isoformat()
                lastModifiedEnd = parser.parse(time_range[1]).isoformat()
                url += '&lastModifiedStart=' + lastModifiedStart + '&lastModifiedEnd=' + lastModifiedEnd
        
        print(url)
        resp = requests.get(url, auth=self.auth)
        
        if not resp.ok:
            raise Exception('Code: {}. Reason: {}. Response: {}'.format(resp.status_code, resp.reason, resp.content))
        
        return resp

license_id = licenses[0]
rest = REST(auth_dict=auth_dict, license_id=license_id)

In [142]:

def _dicts_to_rows(dicts, col_specs) -> List[str]:
    
    title_row = []
    rows = []
        
    for t in dicts:
        row = []
        for i in range(len(col_specs)):
            col_spec = col_specs[i]
            if len(rows) == 0: # its the first row we are dealing with 
                title_row.append(col_spec[0])

            key_name = col_spec[1]
            val = t[key_name]
            if val is None:
                val = ''
            row.append('{}'.format(val))

        rows.append(row)
        
    return [title_row] + rows

class TransferPackages(object):
    
    def __init__(self, transfer_packages: List[Dict]) -> None:
        self._packages = transfer_packages
        
    def to_rows(self) -> List[str]:
        col_specs = [
            ('Package Id', 'PackageId'),
            ('Package', 'PackageLabel'),
            ('Package Type', 'PackageType'),
            ('Item', 'ProductName'),
            ('Item Category', 'ProductCategoryName'),
            ('Item Strain Name', 'ItemStrainName'),
            ('Item State', 'ShipmentPackageState'),
            ('Received Qty', 'ReceivedQuantity'),
            ('UoM', 'ReceivedUnitOfMeasureName'),
            ('Item Unit Qty', 'ItemUnitQuantity'),
            ('Item Unit Weight', 'ItemUnitWeight'),
            ('Is Testing Sample', 'IsTestingSample')
            # ReceiverDollarAmount
        ]
        return _dicts_to_rows(self._packages, col_specs)

class Transfers(object):
    
    def __init__(self, transfers: List[Dict]) -> None:
        self._transfers = transfers
    
    @staticmethod
    def build(transfers: List[Dict]) -> 'Transfers':
        return Transfers(transfers)
    
    def get_delivery_ids(self) -> List[str]:
        return [t['DeliveryId'] for t in self._transfers]
    
    def to_rows(self) -> List[str]:
        col_specs = [
            ('Manifest', 'ManifestNumber'),
            ('Origin Lic', 'ShipperFacilityLicenseNumber'),
            ('Origin Facility', 'ShipperFacilityName'),
            # Origin Facility Type
            ('Dest Lic', 'RecipientFacilityLicenseNumber'),
            ('Destination Facility', 'RecipientFacilityName'),
            ('Type', 'ShipmentTypeName'),
            ('Received', 'ReceivedDateTime'),
            ('Num Packages', 'PackageCount')
        ]
        
        return _dicts_to_rows(self._transfers, col_specs)


In [ ]:
#resp = rest.get('/facilities/v1')
#resp = rest.get('/sales/v1/receipts/active')
#resp = rest.get('/transfers/v1/types')
resp = rest.get('/transfers/v1/incoming', time_range=['04/02/2021'])
transfers = json.loads(resp.content)
print('Found {} transfers'.format(len(transfers)))

In [112]:

transfers_obj = Transfers.build(transfers)
    
for row in transfers_obj.to_rows():
    print(row)
    
transfer_ids = transfers_obj.get_delivery_ids()

['Manifest', 'Origin Lic', 'Origin Facility', 'Dest Lic', 'Destination Facility', 'Type', 'Received', 'Num Packages']
['0001756041', 'C11-0001005-LIC', 'HUMBOLDT SEED COMPANY, LLC', 'CCL19-0001462', 'Kalifornia Green Akres', 'Transfer', '2021-04-02T17:53:59+00:00', 1]
['0001732169', 'CCL19-0005494', 'Half Moon Grow Nursery Inc', 'CCL19-0001462', 'Kalifornia Green Akres', 'Transfer', '2021-04-02T17:54:07+00:00', 13]


In [102]:
# Want to ignore testing license transfers (why they want to look at type)
transfer_ids
#rest.get('/packages/v1/0001756041')

[1757041, 1733169]

In [132]:
transfer_id = transfer_ids[1]
resp = rest.get(f'/transfers/v1/delivery/{transfer_id}/packages')
t_packages_json = json.loads(resp.content)

https://api-ca.metrc.com/transfers/v1/delivery/1733169/packages?licenseNumber=CCL19-0001462


In [143]:
transfer_packages = TransferPackages(t_packages_json)
transfer_packages.to_rows()

[['Package Id',
  'Package',
  'Package Type',
  'Item',
  'Item Category',
  'Item Strain Name',
  'Item State',
  'Received Qty',
  'UoM',
  'Item Unit Qty',
  'Item Unit Weight',
  'Is Testing Sample'],
 ['12747498',
  '1A406030000F873000028206',
  'ImmaturePlant',
  'Grand Daddy Purple Premium Clone',
  'Clone - Cutting',
  '',
  'Accepted',
  '50.0',
  'Each',
  '',
  '',
  'False'],
 ['12747499',
  '1A406030000F873000028207',
  'ImmaturePlant',
  'Grand Daddy Purple Premium Clone',
  'Clone - Cutting',
  '',
  'Accepted',
  '50.0',
  'Each',
  '',
  '',
  'False'],
 ['12747500',
  '1A406030000F873000028208',
  'ImmaturePlant',
  'Grand Daddy Purple Premium Clone',
  'Clone - Cutting',
  '',
  'Accepted',
  '50.0',
  'Each',
  '',
  '',
  'False'],
 ['12748401',
  '1A406030000F873000028209',
  'ImmaturePlant',
  'Grand Daddy Purple Premium Clone',
  'Clone - Cutting',
  '',
  'Accepted',
  '50.0',
  'Each',
  '',
  '',
  'False'],
 ['12748402',
  '1A406030000F873000028210',
  'Imm

In [134]:
t_packages_json

[{'PackageId': 12747498,
  'PackageLabel': '1A406030000F873000028206',
  'PackageType': 'ImmaturePlant',
  'SourceHarvestNames': '',
  'SourcePackageLabels': '',
  'ProductName': 'Grand Daddy Purple Premium Clone',
  'ProductCategoryName': 'Clone - Cutting',
  'ItemStrainName': None,
  'ItemUnitCbdPercent': None,
  'ItemUnitCbdContent': None,
  'ItemUnitCbdContentUnitOfMeasureName': None,
  'ItemUnitCbdContentDose': None,
  'ItemUnitCbdContentDoseUnitOfMeasureName': None,
  'ItemUnitThcPercent': None,
  'ItemUnitThcContent': None,
  'ItemUnitThcContentUnitOfMeasureName': None,
  'ItemUnitThcContentDose': None,
  'ItemUnitThcContentDoseUnitOfMeasureName': None,
  'ItemUnitVolume': None,
  'ItemUnitVolumeUnitOfMeasureName': None,
  'ItemUnitWeight': None,
  'ItemUnitWeightUnitOfMeasureName': None,
  'ItemServingSize': '',
  'ItemSupplyDurationDays': None,
  'ItemUnitQuantity': None,
  'ItemUnitQuantityUnitOfMeasureName': None,
  'LabTestingState': 'NotRequired',
  'ProductionBatchNumber'